In [1]:
import pandas as pd
import os
import pluto_parquet as pp
from zips import unzip_to_temp
from shutil import rmtree
from functools import partial
import warnings
from google.cloud import storage
import tempfile
import logging
from logging.config import fileConfig

In [2]:
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'etc/secret.json'

In [3]:
loginipath='logs/logging_config.ini'
fileConfig(loginipath, defaults={'logfilename': 'logs/pluto.log'})
logger = logging.getLogger('sLogger')

In [4]:
client = storage.Client()
bucket = client.bucket('raw-pluto')

In [5]:
zip_links=pd.read_csv("etc/zip_links.csv")[18:]

In [6]:
for _, row in zip_links.iterrows():
    
    pluto_year=str(row.year)
    zip_path=row.path
    
    tmp_dir=unzip_to_temp(zip_path)
    paths=pp.list_shp(tmp_dir)
    root_paths=['..' + p for p in paths]
    
    aggregate_year = partial(pp.agg_from_path, year=pluto_year)
    gdf_list=list(map(aggregate_year, root_paths))
    df=pd.concat(gdf_list)
    gdf_list=None
    
    
    rmtree(tmp_dir)
    
    
    
    with tempfile.NamedTemporaryFile() as fp:
        df.to_parquet(fp)
        df=None
        blob_name=f'pluto_{pluto_year}.parquet'
        logger.info(f'loading making blob: {blob_name}')
        blob=bucket.blob(blob_name)
        blob.upload_from_filename(fp.name)
        logger.info(f'succesfully loaded df with rows: {df.shape[0]} and columns: {df.shape[1]} to file: {blob_name}')